In [90]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [156]:
from sklearn.metrics import accuracy_score,mean_absolute_error,f1_score
import pickle

In [91]:
df = pd.read_csv('cleaned_data.csv')

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3986 entries, 0 to 3985
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Author_website  3986 non-null   object
 1   Headline        3986 non-null   object
 2   Body            3986 non-null   object
 3   Label           3986 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 124.7+ KB


In [93]:
df.head()

,Author_website,Headline,Body,Label
0,BEFOREITNEWS,nick folks days tampa numbered buccaneers bri...,nick folks days tampa numbered buccaneers bri...,0
1,BEFOREITNEWS,california doubles down stupidity is now a sa...,california doubles down stupidity is now a sa...,0
2,BEFOREITNEWS,where is president trumps sense of compassion,a potato battery can light room over month di...,0
3,BEFOREITNEWS,report jets sign running backreturn man trava...,report jets sign running backreturn man trava...,0
4,BEFOREITNEWS,jake arrieta say feel good ahead game 4 start...,jake arrieta say feel good ahead game 4 start...,0


In [94]:
df.tail()

,Author_website,Headline,Body,Label
3981,ACTIVISTPOST,us missile machinations undoes nonproliferati...,by ulson gunnarwhen come nuclear weapon upon ...,0
3982,BEFOREITNEWS,929 through 40 hbd cannonball paul camnitz cl...,red flag warning these california wildfires a...,0
3983,BEFOREITNEWS,107 chuck axed hbd brickyard adam moonlight a...,an embattled pharmaceutical company that sell...,0
3984,ACTIVISTPOST,msm ignores videos of syrians celebrating saa...,by brandon turbevillewhile western medium lam...,0
3985,BBC,facebook confirms russia ad instagram,image copyright getty imagesfacebook confirme...,1


In [95]:
df = df.sample(frac=1,random_state=24).reset_index()
df.drop(columns = ['index'],axis =1,inplace=True)

In [96]:
df.head()

,Author_website,Headline,Body,Label
0,REUTERS,accor 920 million mantra bid australia touris...,a sign bearing logo mantra group ltd displaye...,1
1,BBC,blade runner 2049 disappoints us box office,image copyright getty images image caption ry...,1
2,BEFOREITNEWS,shrimpton kerry cassidy in ireland 16 sept 17...,streamed live 9 hour agomirjam jansekerry cas...,0
3,REUTERS,apple name former honeywell executive new gen...,a security officer apple store staff seen ope...,1
4,CNN,how ferraris first female driver disappear,written jared zaugg cnnas one greatest female...,1


In [97]:
count_vectorizer = CountVectorizer()
tfid_transform = TfidfTransformer()

In [98]:
def transform_vectorize(sample):
    count_vectorizer.fit(sample)
    temp = count_vectorizer.fit_transform(sample)
    tfid_transform.fit(temp)
    mat = term_frequency_body.fit_transform(temp).todense()
    return mat

In [99]:
df['Total_words'] = df['Author_website']+ ':'+df['Headline']+''+df['Body']


In [100]:
df.head(1)

,Author_website,Headline,Body,Label,Total_words
0,REUTERS,accor 920 million mantra bid australia touris...,a sign bearing logo mantra group ltd displaye...,1,REUTERS: accor 920 million mantra bid australi...


In [101]:
x_mat = transform_vectorize(df['Total_words'])

In [102]:
x_mat.shape

(3986, 84601)

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
x_train,x_test,y_train,y_test = train_test_split(x_mat,df['Label'].values,test_size = 0.2,random_state = 24)

In [105]:
x_train.shape, y_train.shape

((3188, 84601), (3188,))

# Logistic Regression

In [106]:
from sklearn.linear_model import LogisticRegressionCV

In [107]:
logistic_model = LogisticRegressionCV(Cs = 10,verbose=10)

In [108]:
logistic_model.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  6.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  7.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  7.6min finished


LogisticRegressionCV(verbose=10)

In [109]:
trainpredict_logistic = logistic_model.predict(x_train)

In [110]:
trainscore_log_model = accuracy_score(y_train,trainpredict_logistic)

In [111]:
testpredict_logistic = logistic_model.predict(x_test)

In [112]:
testscore_log_model = accuracy_score(y_test,testpredict_logistic)

In [113]:
log_scores = pd.DataFrame({'Training score':trainscore_log_model,'Testing score':testscore_log_model},index=[0])
log_scores

,Training score,Testing score
0,1.0,0.986216


In [150]:
logistic_f1score = f1_score(y_test,testpredict_forest)
logistic_f1score

0.9796472184531886

In [114]:
logistic_model.Cs_ , logistic_model.C_

(array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
        3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
        1.29154967e+03, 1.00000000e+04]),
 array([1291.54966501]))

   # Decision Tree

In [115]:
from sklearn.tree import DecisionTreeClassifier

In [116]:
tree_model = DecisionTreeClassifier(random_state = 24,criterion='entropy')

In [117]:
from sklearn.model_selection import GridSearchCV

In [118]:
param_dict = {'max_depth':[75,76,77,78,80]}

In [122]:
search_cv = GridSearchCV(tree_model,param_dict,verbose=10)

In [123]:
search_cv.fit(x_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] max_depth=75 ....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........................ max_depth=75, score=0.998, total=  16.9s
[CV] max_depth=75 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.1s remaining:    0.0s


[CV] ........................ max_depth=75, score=0.992, total=  16.7s
[CV] max_depth=75 ....................................................


...

[CV] ........................ max_depth=76, score=1.000, total=  15.6s
[CV] max_depth=76 ....................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.6min remaining:    0.0s


[CV] ........................ max_depth=76, score=0.997, total=  16.7s
[CV] max_depth=77 ....................................................
[CV] ........................ max_depth=77, score=0.998, total=  16.8s
[CV] max_depth=77 ....................................................
[CV] ........................ max_depth=77, score=0.992, total=  17.1s
[CV] max_depth=77 ....................................................
[CV] ........................ max_depth=77, score=0.992, total=  16.0s
[CV] max_depth=77 ....................................................
[CV] ........................ max_depth=77, score=1.000, total=  17.5s
[CV] max_depth=77 ....................................................
[CV] ........................ max_depth=77, score=0.997, total=  17.5s
[CV] max_depth=78 ....................................................
[CV] ........................ max_depth=78, score=0.998, total=  17.1s
[CV] max_depth=78 ....................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  7.1min finished


GridSearchCV(estimator=DecisionTreeClassifier(criterion='entropy',
                                              random_state=24),
             param_grid={'max_depth': [75, 76, 77, 78, 80]}, verbose=10)

In [125]:
search_cv.best_params_

{'max_depth': 75}

In [126]:
tree_model.max_depth = 75

In [127]:
tree_model.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=75, random_state=24)

In [128]:
trainpredict_tree  = tree_model.predict(x_train)

In [129]:
testpredict_tree = tree_model.predict(x_test)
testpredict_tree[0:5]

array([0, 1, 0, 1, 1], dtype=int64)

In [130]:
trainscore_tree = accuracy_score(y_train,trainpredict_tree)
testscore_tree = accuracy_score(y_test,testpredict_tree)

In [131]:
tree_scores = pd.DataFrame({'Training score':trainscore_tree,'Testing score':testscore_tree},index=[0])
tree_scores

,Training score,Testing score
0,1.0,0.997494


In [149]:
tree_f1score = f1_score(y_test,testpredict_tree)
tree_f1score

0.997229916897507

In [132]:
#

In [133]:
# We will try Random Forest model to overcome overfitting

# Random Forest Model

In [134]:
from sklearn.ensemble import RandomForestClassifier

In [135]:
forest_clf = RandomForestClassifier(random_state= 24,verbose=10)

In [136]:
params = {'max_depth':[73,74,75,76],'n_estimators':[250,275,300]}

In [137]:
search_cv = GridSearchCV(forest_clf,params,verbose=10)

In [138]:
search_cv.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] max_depth=73, n_estimators=250 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 250


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 2 of 250


...

building tree 9 of 300


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.2s remaining:    0.0s


building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of 300
building tree 49 of 300
building tree 50 of 300
building tree 51

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  1.3min finished


GridSearchCV(estimator=RandomForestClassifier(random_state=24, verbose=10),
             param_grid={'max_depth': [73, 74, 75, 76],
                         'n_estimators': [250, 275, 300]},
             verbose=10)

In [139]:
search_cv.best_params_

{'max_depth': 75, 'n_estimators': 300}

In [140]:
forest_clf.max_depth = 75
forest_clf.n_estimators = 300

In [141]:
forest_clf.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 300


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 300


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


...

building tree 9 of 300


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.2s remaining:    0.0s


building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of 300
building tree 49 of 300
building tree 50 of 300
building tree 51

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  1.3min finished


RandomForestClassifier(max_depth=75, n_estimators=300, random_state=24,
                       verbose=10)

In [142]:
trainpredict_forest = forest_clf.predict(x_train)
testpredict_forest = forest_clf.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elap

In [143]:
testpredict_forest[0:10]

array([0, 1, 0, 1, 1, 0, 0, 0, 1, 0], dtype=int64)

In [145]:
trainscore_forest = accuracy_score(y_train,trainpredict_forest)
testscore_forest = accuracy_score(y_test,testpredict_forest)

In [146]:
forest_scores = pd.DataFrame({'Training score':trainscore_forest,'Testing score':testscore_forest},index=[0])
forest_scores

,Training score,Testing score
0,1.0,0.981203


In [151]:
forest_f1score = f1_score(y_test,testpredict_forest)
forest_f1score

0.9796472184531886

In [152]:
#

# Comparing all the models

In [153]:
compare_models = pd.DataFrame({'Model':['Logistic Regression','Decision Tree','Random Forest'],\
                               'Accuracy_score':[testscore_log_model,testscore_tree,testscore_forest],\
                               'F1_Score':[logistic_f1score,tree_f1score,forest_f1score]})

In [154]:
compare_models

,Model,Accuracy_score,F1_Score
0,Logistic Regression,0.986216,0.979647
1,Decision Tree,0.997494,0.997230
2,Random Forest,0.981203,0.979647


In [155]:
#

# Pickling

In [157]:
pickle.dump(tree_model,open('model_clf.pkl','wb'))

In [158]:
pickle.dump(count_vectorizer,open('vectorizer.pkl','wb'))
pickle.dump(tfid_transform,open('tfid_transformer.pkl','wb'))